## PROJECT SCENARIO

In this times series project, we will build a model that predicts the unit sales for thousands of items sold at Corporation Favorita, a large Ecuadorian-based grocery retailer.

## PROJECT DESCRIPTION

In this project, we will analyze past sales data from Corporation Favorita. Also, we will analyze other occurrences that may have impact on the sales performance of the retail store chains.

Also, we will build different machine learning models, evaluate the models, test the models, compare performances of the models and select the model that best predicts future grocery sales.

## HYPOTHESIS

## BUSINESS QUESTIONS

1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

#### ADDITIONAL QUESTION
8. Is there a significant difference in sales during holidays versus non-holidays?
9. Do stores located in certain locales have higher holiday sales compared to others?
10. Do stores in certain cities or states have higher sales compared to others?
11. Is there a difference in transactions during holidays versus non-holidays?
12. Does the promotion of certain items during holidays impact sales?

### IMPORT LIBRARIES FOR PROJECT

In [5]:
import numpy as np
import pandas as pd
import matplotlib
from datetime import datetime
from matplotlib import pyplot as plt
import seaborn as sns

from scipy import interpolate

import os, pickle
import sklearn


import warnings
warnings.filterwarnings("ignore")


%matplotlib inline

### IMPORT PROJECT DATASETS

In [31]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
stores = pd.read_csv("stores.csv")
holidays = pd.read_csv("holidays_events.csv")
oil = pd.read_csv("oil.csv")
transactions = pd.read_csv("transactions.csv")
sample = pd.read_csv("sample_submission.csv")


## EXPLORATORY DATA ANALYSIS

#### EXPLORING THE TRAIN DATASET

In [28]:
# Checking the head and tail of the train data

train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [7]:
# Checking the size of the train dataset

train.shape

(3000888, 6)

The train dataset has 3000888 rows and 6 colums

In [15]:
# Checking the basic info of the train data

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


The date column is recorded as object. It has to be converted to date format

In [16]:
# Checking for null values in the train data

train.isnull().any()

id             False
date           False
store_nbr      False
family         False
sales          False
onpromotion    False
dtype: bool

 There are no null values in the train dataset

In [17]:
# Checking for missing data in the train dataset

train.duplicated().any()

False

There are no duplicated values in the train dataset

In [20]:
# Checking the basic statistics of the train data

train.describe(include="all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,3000888.0,NaN,NaN,NaN,1500443.5,866281.891642,0.0,750221.75,1500443.5,2250665.25,3000887.0
date,3000888,1684,2013-01-01,1782,NaN,NaN,NaN,NaN,NaN,NaN,NaN
store_nbr,3000888.0,NaN,NaN,NaN,27.5,15.585787,1.0,14.0,27.5,41.0,54.0
family,3000888,33,AUTOMOTIVE,90936,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sales,3000888.0,NaN,NaN,NaN,357.775749,1101.997721,0.0,0.0,11.0,195.84725,124717.0
onpromotion,3000888.0,NaN,NaN,NaN,2.60277,12.218882,0.0,0.0,0.0,0.0,741.0


There are 33 different families of grocery products. Automotive family has the most number of products.<br />Also, there are 54 stores in the data

#### EXPLORING THE TEST DATASET

In [27]:
# Checking the head and tail of the test data

test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [23]:
# Checking the sahpe of the test dataset

test.shape

(28512, 5)

There are 2852 rows and 5 columns. It is noted that the sales column has been removed form the test data

In [24]:
# Checking the basic info of the test data

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


There are no null values, the date column is recorded as an pbject.

In [25]:
# Checking for duplicates in the test data

test.duplicated().any()

False

There are no duplicated values in the test data

In [30]:
# Checking for unique values in the test data

test.nunique()

id             28512
date              16
store_nbr         54
family            33
onpromotion      212
dtype: int64

There are 16 days recorded, 54 stores and 33 different product families

#### EXPLORING THE STORES DATA

In [33]:
# Checking the head  of the stores data

stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [41]:
# Checking the shape of the stores data

stores.shape

(54, 5)

There are 54 rows and 5 columns in the stores data

In [34]:
# Checking the basic info of the stores data

stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


There are no null entries in the stores data

In [36]:
# Checking for dupliated entries in the stores data

stores.duplicated().any()

False

There are no duplicated entries in the stores data

In [38]:
# Checking for unique values in the stores data

stores.nunique()

store_nbr    54
city         22
state        16
type          5
cluster      17
dtype: int64

Stores are located at 22 different Cities in 16 States. There are 5 different types of stores.There are 17 similar stores located in these cities.

#### EXPLORING THE HOLIDAYS DATA

In [54]:
# Checking the head of the holidays data
pd.set_option('display.max_rows', None)
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
6,2012-06-23,Holiday,Local,Guaranda,Cantonizacion de Guaranda,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
8,2012-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
9,2012-06-25,Holiday,Local,Machala,Fundacion de Machala,False


In [45]:
# Checking the basic info of the holidays data

holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


The date column is recorded as object. This will be changed to the date format. There are no null values in the data.

In [46]:
# Checking for duplicated values in the data

holidays.duplicated().any()

False

There are no duplicated values in the holidays dataset

In [47]:
# Checking the number of unique values in the holidays dataset

holidays.nunique()

date           312
type             6
locale           3
locale_name     24
description    103
transferred      2
dtype: int64

There are 6 types of holidays and 2 forms of holidays transfers

In [51]:
# Checking the different types of holidays

list(holidays.type.unique())

['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event']

In [61]:
# Checking holidays that were transferred

holidays[holidays.transferred == True]

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


These holidays were transfered to be celebrated on another dates.

In [65]:
# Checking corresponding dates on which holidays were transferred to

holidays[holidays.type == "Transfer"]

,date,type,locale,locale_name,description,transferred
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
73,2013-10-11,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
136,2014-10-10,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
256,2016-05-27,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
265,2016-07-24,Transfer,Local,Guayaquil,Traslado Fundacion de Guayaquil,False
269,2016-08-12,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
298,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,False
304,2017-04-13,Transfer,Local,Cuenca,Fundacion de Cuenca,False
313,2017-05-26,Transfer,National,Ecuador,Traslado Batalla de Pichincha,False
325,2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False


These dates were had holidays transfered to them and were observed

In [66]:
# Checking additional holidays

holidays[holidays.type == "Additional"]

,date,type,locale,locale_name,description,transferred
28,2012-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
33,2012-12-22,Additional,National,Ecuador,Navidad-3,False
34,2012-12-23,Additional,National,Ecuador,Navidad-2,False
36,2012-12-24,Additional,National,Ecuador,Navidad-1,False
38,2012-12-26,Additional,National,Ecuador,Navidad+1,False
40,2012-12-31,Additional,National,Ecuador,Primer dia del ano-1,False
53,2013-05-11,Additional,National,Ecuador,Dia de la Madre-1,False
64,2013-07-24,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False
81,2013-12-05,Additional,Local,Quito,Fundacion de Quito-1,False


These days were added to regular calendar holidays to have longer periods of holidays

In [67]:
# Checking  Bridge holidays

holidays[holidays.type == "Bridge"]

,date,type,locale,locale_name,description,transferred
35,2012-12-24,Bridge,National,Ecuador,Puente Navidad,False
39,2012-12-31,Bridge,National,Ecuador,Puente Primer dia del ano,False
156,2014-12-26,Bridge,National,Ecuador,Puente Navidad,False
160,2015-01-02,Bridge,National,Ecuador,Puente Primer dia del ano,False
277,2016-11-04,Bridge,National,Ecuador,Puente Dia de Difuntos,False


These were days that were added to regular holidays to extend the break across a long weekend

In [68]:
# Checking for Work days

holidays[holidays.type == "Work Day"]

,date,type,locale,locale_name,description,transferred
42,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
43,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
149,2014-12-20,Work Day,National,Ecuador,Recupero Puente Navidad,False
161,2015-01-10,Work Day,National,Ecuador,Recupero Puente Primer dia del ano,False
283,2016-11-12,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


These days were not working days but were observed as working days to payback for the bridge holidays

#### EXPLORING THE OIL DATA

In [93]:
# Checking the head of the oil data

oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [96]:
# Checking the shape of the oil data

oil.shape

(1218, 2)

There are 1218 ros and 2 columns

In [97]:
# Checking the basic info of the oil data

oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


The date is in object format. It must corrected to date format. There are null 

In [99]:
# Checking the null values

oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

There are 43 null values in the oil data. Since this a crude oil price,, there is no possibilty that crude oil price will 0. Hence, we will compute it with either the mean, minimum value or 50th percentile

In [100]:
# Cecking for duplicated values in the oil data

oil.duplicated().any()

False

There are no duplicated values in the oil data

In [104]:
# Checking the basic statistics of the oil data

oil.describe(exclude="object").transpose()

,count,mean,std,min,25%,50%,75%,max
dcoilwtico,1175.0,67.714366,25.630476,26.19,46.405,53.19,95.66,110.62


#### EXPLORING THE TRANSACTIONS DATASET

In [112]:
# Checking the head of the transactions data

transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [113]:
# Checking the shape of the transactions data

transactions.shape

(83488, 3)

There are 834888 rows and 3 columns in the transactions data

In [107]:
# Checking the basic info of the transactions data

transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


The date is not in the correct format. It is recorded in object. We will convert it to date format. There are no null values in the data.

In [109]:
# Checking for duplicated values in the transactions data

transactions.duplicated().any()

False

There are no duplicated values in the data.